In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import models, layers
import cv2
import os
from os import listdir
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from skimage.util import random_noise
import pandas as pd
import xgboost as xgb
import glob
from tensorflow.keras.models import Model
from tensorflow.keras import layers, losses
import seaborn as sns

In [ ]:
BATCH_SIZE = 32
IMAGE_SIZE = 256
CHANNELS = 3
EPOCHS=100
N_IMAGES = 200
LR = 0.001

In [ ]:
root_dir = './PlantVillage-Dataset/segmented/'

In [ ]:
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None:
            image = cv2.resize(image, (IMAGE_SIZE,IMAGE_SIZE))   
            return img_to_array(image)
        else:
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None

In [ ]:
image_list, label_list = [], []

try:
    print("[INFO] Loading images ...")
    plant_disease_folder_list = listdir(root_dir)

    for plant_disease_folder in plant_disease_folder_list:
        print(f"[INFO] Processing {plant_disease_folder} ...")
        plant_disease_image_list = listdir(f"{root_dir}/{plant_disease_folder}/")

        for image in plant_disease_image_list[:N_IMAGES]:
            image_directory = f"{root_dir}/{plant_disease_folder}/{image}"
            if image_directory.endswith(".jpg")==True or image_directory.endswith(".JPG")==True:
                image_list.append(convert_image_to_array(image_directory))
                label_list.append(plant_disease_folder)

    print("[INFO] Image loading completed")  
except Exception as e:
    print(f"Error : {e}")

# Transform the loaded training image data into numpy array
np_image_list = np.array(image_list, dtype=np.float32) / 255.0
print()

# Check the number of images loaded for training
image_len = len(image_list)
print(f"Total number of images: {image_len}")

In [ ]:
label_binarizer = LabelBinarizer()
image_labels = label_binarizer.fit_transform(label_list)

n_classes = len(label_binarizer.classes_)

print("Total number of classes: ", n_classes)

In [ ]:
print("[INFO] Splitting data to train and test...")
x_train, x_test, y_train, y_test = train_test_split(np_image_list, image_labels, test_size=0.2, random_state = 42)

In [ ]:
from keras.applications.resnet_v2 import ResNet50V2

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

augment = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
                             height_shift_range=0.1, shear_range=0.2, 
                             zoom_range=0.2, horizontal_flip=True, 
                             fill_mode="nearest")

In [ ]:
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D
from keras.layers import Dense
from keras.layers import Dropout
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import Adam

baseModel = ResNet50V2(include_top=False, input_shape=(IMAGE_SIZE, IMAGE_SIZE, CHANNELS))
for layer in baseModel.layers:
    layer.trainable = False

ResNet50V2_model = Sequential()
ResNet50V2_model.add(baseModel)
ResNet50V2_model.add(GlobalAveragePooling2D())
ResNet50V2_model.add(Dense(512, activation="sigmoid"))
ResNet50V2_model.add(Dropout(0.5))
ResNet50V2_model.add(Dense(n_classes, activation='softmax'))

opt = Adam(learning_rate=LR)

ResNet50V2_model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=opt)

In [ ]:
ResNet50V2_model.summary()

In [ ]:
history = ResNet50V2_model.fit(augment.flow(x_train, y_train),
                              batch_size=BATCH_SIZE, 
                              validation_data=(x_test, y_test),
                              steps_per_epoch=len(x_train) // BATCH_SIZE,
                              epochs=EPOCHS, 
                              verbose=1)

In [ ]:
ResNet50V2_model.evaluate(x_test, y_test)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
feature_extractor = ResNet50V2_model.predict(x_train)

In [ ]:
features = feature_extractor.reshape(feature_extractor.shape[0], -1)
print(features.shape)
X_for_training = features
print(X_for_training.shape)

In [ ]:
y_train = label_binarizer.inverse_transform(y_train)
y_test = label_binarizer.inverse_transform(y_test)

In [ ]:
y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y_train)
y_train_encoded = le.transform(y_train)
le.fit(y_test)
y_test_encoded = le.transform(y_test)

In [ ]:
XGB_model = xgb.XGBClassifier(learning_rate='0.01', n_estimators=500)
XGB_model.fit(X_for_training, y_train_encoded)

In [ ]:
X_test_feature = ResNet50V2_model.predict(x_test)

In [ ]:
X_test_features = X_test_feature.reshape(X_test_feature.shape[0], -1)

In [ ]:
prediction = XGB_model.predict(X_test_features)
prediction = le.inverse_transform(prediction)

In [ ]:
from sklearn import metrics
print ("Accuracy = ", metrics.accuracy_score(y_test, prediction))

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

cm = confusion_matrix(y_test, prediction)
print(cm)
sns.heatmap(cm, annot=True)

In [ ]:
print(x_test.shape)

In [ ]:
n=np.random.randint(0, x_test.shape[0])
print(n)

In [ ]:
img = x_test[n]
img.shape

In [ ]:
n=np.random.randint(0, x_test.shape[0])
img = x_test[n]
plt.imshow(img)
input_img = np.expand_dims(img, axis=0) 
input_img_feature = ResNet50V2_model.predict(input_img)
input_img_features = input_img_feature.reshape(input_img_feature.shape[0], -1)
prediction = XGB_model.predict(input_img_features)[0] 
prediction = le.inverse_transform([prediction])  
print("The prediction for this image is: ", prediction)
print("The actual label for this image is: ", y_test[n])